Baseline Deteksi Ujaran Kebencian Metode CNN
Copyright@ I Gde Bagus Janardana Abasan

In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, Bidirectional, AveragePooling1D, LSTM, GRU, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D, CuDNNGRU , GlobalAveragePooling1D, Concatenate, TimeDistributed
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta, Adamax, Nadam
from keras import regularizers
import tensorflow as tf
import matplotlib.pyplot as plt

## CNN MODEL BASELINE
1. Tanpa TF-IDF ( CountVectorize )
2. Testing Test Size ( splitting )

Expected Result :
1. test-size yang bagus untuk model ini.

In [9]:
# Menampilkan informasi perangkat GPU yang digunakan
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    device_details = tf.config.experimental.get_device_details(device)
    print(device_details)

try:
    print(f'Found {physical_devices[0]} GPU(s)')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    assert tf.config.experimental.get_memory_growth(physical_devices[0])
    print('Memory growth set to True, using Nvidia Tesla T4')
except:
    print('Failed to set memory growth to True')
    pass

{'compute_capability': (7, 5), 'device_name': 'Tesla T4'}
Found PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') GPU(s)
Memory growth set to True, using Nvidia Tesla T4


In [10]:
def process_data(test_size, max_features):
  dataset = pd.read_csv('../../data/data_preprocessed/dataset/DatasetHateSpeech_Final_TA2023.csv', usecols=['preprocess_final', 'preprocess_token', 'label_final'])
  # make data['label_fase_1'] to 0 and 1 binary classifier karena tensor hanya bisa input 0 =< label =< 1
  dataset['label_final'] = dataset['label_final'].apply(lambda x: 1 if x == 'HS' else 0)
  dataset.dropna(inplace=True)

  # feature extration
  # Create a CountVectorizer object and fit it to the training data
  vectorizer = TfidfVectorizer(ngram_range=(1,1), max_features=max_features)
  vectorizer_data = vectorizer.fit_transform(dataset['preprocess_final'])

  # to_pd
  x_final = pd.DataFrame(vectorizer_data.toarray(), columns=vectorizer.get_feature_names_out())
  y_final = dataset['label_final'].copy()

  # split
  X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=test_size, random_state=0)

  return X_train, X_test, y_train, y_test

In [11]:
def create_cnn_model(X_train, X_test, y_train, y_test):
  # reshape
  X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
  X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

  # CNN model
  model = Sequential()
  model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
  model.add(Flatten())
  model.add(Dense(units=32, activation='relu'))
  model.add(Dense(units=1, activation='sigmoid'))
  optimizer = Adam(learning_rate=0.00001)
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  model.build(input_shape=(None, X_train.shape[1], 1))
  model.summary()
  
  with tf.device('/cpu:0'):
    x = tf.convert_to_tensor(X_train, np.float32)
    y = tf.convert_to_tensor(y_train, np.float32)
    
  model.fit(x, y, validation_data=(X_test, y_test), epochs=10, batch_size=64)
  loss = model.history.history['loss']
  val_loss = model.history.history['val_loss']
  print("=== MODEL EVALUATE TEST DATA ===")
  score = model.evaluate(X_test, y_test, batch_size=64, verbose=0)
  print()
  print('Validation Accuracy:', score[1])
  print('Validation Loss:', score[0])
  print()

  return model, loss, val_loss

In [12]:
def create_cnn_model(X_train, X_test, y_train, y_test):
  # reshape
  X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
  X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

  # CNN model
  model = Sequential()
  model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model.add(MaxPooling1D(pool_size=2, strides=2, padding='same'))
  model.add(Flatten())
  model.add(Dense(units=32, activation='relu'))
  model.add(Dense(units=1, activation='sigmoid'))
  optimizer = Adam(learning_rate=0.00001)
  model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
  model.build(input_shape=(None, X_train.shape[1], 1))
  model.summary()
  
  with tf.device('/cpu:0'):
    x = tf.convert_to_tensor(X_train, np.float32)
    y = tf.convert_to_tensor(y_train, np.float32)
    
  model.fit(x, y, validation_data=(X_test, y_test), epochs=10, batch_size=64)
  loss = model.history.history['loss']
  val_loss = model.history.history['val_loss']
  print("=== MODEL EVALUATE TEST DATA ===")
  score = model.evaluate(X_test, y_test, batch_size=64, verbose=0)
  print()
  print('Validation Accuracy:', score[1])
  print('Validation Loss:', score[0])
  print()

  return model, loss, val_loss

In [13]:
def call(test_size, max_features):
  X_train, X_test, y_train, y_test = process_data(test_size, max_features)
  model, loss, val_loss = create_cnn_model(X_train, X_test, y_train, y_test)
   # predict
  y_pred = model.predict(X_test)
  print('using test size :', test_size)
  print()

  # confusion matrix
  classreport = classification_report(y_test, y_pred.round(), digits=4)
  print()
  print(classreport)
  print()
  accscore = accuracy_score(y_test, y_pred.round())
  precscore = precision_score(y_test, y_pred.round(), zero_division=0)
  recscore = recall_score(y_test, y_pred.round())
  f1score = f1_score(y_test, y_pred.round(), average='weighted', zero_division=0)

  return [accscore, precscore, recscore, f1score, loss, val_loss]

In [14]:
test_size = [0.3]
max_features = [5000]
result = []
count = 1
for idx, value_testsize in enumerate(test_size):
  for idx_max, value_features in enumerate(max_features):
    for i in range(5):
      result.append([count] + [value_testsize] + [value_features] + call(value_testsize, value_features))
      count += 1

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 5000, 32)          128       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 2500, 32)         0         
 1D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 80000)             0         
                                                                 
 dense_6 (Dense)             (None, 32)                2560032   
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,560,193
Trainable params: 2,560,193
Non-trainable params: 0
____________________________________________

In [15]:
def loss_func(df):
  for idx, val in enumerate(df['No']):
    plt.plot(df['Loss'].iloc[idx], label=f'Test Size {val}')
    plt.plot(df['Val Loss'].iloc[idx], label=f'Test Size {val}')
    plt.title(f'baseline BiGRU + CNN lfg {df["Test_Size"].iloc[idx]} max_features {df["Max_Features"].iloc[idx]}-{idx}')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right')
    plt.savefig(f'hasil/grafik/bigru+cnn/lfg_baselineCNNBiGRU_testSize{df["Test_Size"].iloc[idx]}-{idx}-max_features{df["Max_Features"].iloc[idx]}.png')
    plt.clf()

In [16]:
df = pd.DataFrame(result, columns=['No', 'Test_Size', 'Max_Features', 'Accuracy Score', 'Precision Score', 'Recall Score', 'F1 Score', 'Loss', 'Val Loss'])
df.to_csv('hasil/baselineCNN-3-5000.csv', index=False)

# print average score by test size
print('Average Score by Test Size')
print('Test Size 0.3 & max features 5000')
print('Average Accuracy Score :', df[(df['Test_Size'] == 0.3) & (df['Max_Features'] == 5000)]['Accuracy Score'].mean())
print('Average Precision Score :', df[(df['Test_Size'] == 0.3) & (df['Max_Features'] == 5000)]['Precision Score'].mean())
print('Average Recall Score :', df[(df['Test_Size'] == 0.3) & (df['Max_Features'] == 5000)]['Recall Score'].mean())
print('Average F1 Score :', df[(df['Test_Size'] == 0.3) & (df['Max_Features'] == 5000)]['F1 Score'].mean())
print()

Average Score by Test Size
Test Size 0.3 & max features 5000
Average Accuracy Score : 0.844312178158229
Average Precision Score : 0.8313071523244908
Average Recall Score : 0.8968098159509204
Average F1 Score : 0.8433339799619046

